#**Integrantes**

  Lucca Costa Lima D'Avila - 171903

  Henrique Casarini Firmino - 180185

  Carlos Eduardo Ferreira da Silva - 223748


#Neste trabalho, iremos demonstrar a habilidade do FNet em atingir resultados comparáveis com um modelo Transformer em um caso de classificação de texto.
#Para isso, utilizaremos um dataset do IMDb com uma coleção de críticas de filmes classificados como positivo ou negativo.
#Para construir o tokenizer, modelo etc. usaremos componentes do KerasNLP.

In [ ]:
!pip install keras_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 44.5 MB/s 
     |████████████████████████████████| 588.3 MB 20 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 5.9 MB 54.5 MB/s 
     |████████████████████████████████| 578.1 MB 8.0 kB/s 
     |████████████████████████████████| 578.1 MB 7.2 kB/s 
     |████████████████████████████████| 4.6 MB 63.7 MB/s 


In [ ]:
import keras_nlp
import random
import tensorflow as tf
import os

from tensorflow import keras
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

keras.utils.set_random_seed(42)

#Embedding A

Parâmetros

## Sequence Length Pequeno

In [ ]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 512
VOCAB_SIZE = 15000

EMBED_DIM = 128
INTERMEDIATE_DIM = 512

#Carregando o dataset.
O dataset usado é do IMDb

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz

--2022-12-02 15:56:41--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  23.5MB/s    in 3.4s    

2022-12-02 15:56:45 (23.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
print(os.listdir("./aclImdb"))
print(os.listdir("./aclImdb/train"))
print(os.listdir("./aclImdb/test"))

['imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']
['pos', 'urls_pos.txt', 'unsup', 'labeledBow.feat', 'urls_unsup.txt', 'unsupBow.feat', 'neg', 'urls_neg.txt']
['pos', 'urls_pos.txt', 'labeledBow.feat', 'neg', 'urls_neg.txt']


#O diretório contém dois sub-diretórios (train e test). Cada um deles possui duas pastas (pos e neg) para críticas positivas e negativas. Antes de carregar o dataset, vamos deletar a pasta train/unsup, já que ela possui apenas críticas sem classificação.

In [ ]:
!rm -rf aclImdb/train/unsup

#Vamos usar o "keras.utils.text_dataset_from_directory" para gerar o dataset rotulado.

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42,
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42,
)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=BATCH_SIZE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


#Convertendo o texto para minúsculas

In [ ]:
train_ds = train_ds.map(lambda x, y: (tf.strings.lower(x), y))
val_ds = val_ds.map(lambda x, y: (tf.strings.lower(x), y))
test_ds = test_ds.map(lambda x, y: (tf.strings.lower(x), y))

#Imprimindo alguns exemplos

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'an illegal immigrant resists the social support system causing dire consequences for many. well filmed and acted even though the story is a bit forced, yet the slow pacing really sets off the conclusion. the feeling of being lost in the big city is effectively conveyed. the little person lost in the big society is something to which we can all relate, but i cannot endorse going out of your way to see this movie.'
0
b"to get in touch with the beauty of this film pay close attention to the sound track, not only the music, but the way all sounds help to weave the imagery. how beautifully the opening scene leading to the expulsion of gino establishes the theme of moral ambiguity! note the way music introduces the characters as we are led inside giovanna's marriage. don't expect to find much here of the political life of italy in 1943. that's not what this is about. on the other hand, if you are susceptible to the music of images and sounds, you will be led into a word that reaches beyond

#Tokenizando os dados
Utilizaremos o keras_nlp.tokenizers.WordPieceTokenizer para fazer esse processo. Ele recebe um vocabulário WordPiece e possui funções para tokenizar o texto, e destokenizar uma sequência de tokens.

In [ ]:
def train_word_piece(ds, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # Tamanho do vocabulário de destino
        vocab_size=vocab_size,
        # Tokens reservados que devem ser incluídos no vocabulário 
        reserved_tokens=reserved_tokens,
        # Argumentos para `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    # Extrai amostras de texto (remove os rótulos).
    word_piece_ds = ds.unbatch().map(lambda x, y: x)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab

In [ ]:
reserved_tokens = ["[PAD]", "[UNK]"]
train_sentences = [element[0] for element in train_ds]
vocab = train_word_piece(train_ds, VOCAB_SIZE, reserved_tokens)

#Imprimindo alguns tokens:

In [ ]:
print("Tokens: ", vocab[100:110])

Tokens:  ['in', 'this', 'that', 'was', 'as', 'for', 'movie', 'with', 'but', 'film']


#Definindo o tokenizer
Vamos configurar o tokenizer com os vocabulários treinados anteriormente. Vamos definir um tamanho máximo para que todas sequências fiquem com o mesmo tamanho, se o tamanho da sequência for menor que o especificado. Se não, a sequência é truncada.

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

Tokenizando um exemplo do dataset para verificar se o texto foi tokenizado corretamente.

In [ ]:
input_sentence_ex = train_ds.take(1).get_single_element()[0][0]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Frase: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Texto recuperado após a destokenização: ", tokenizer.detokenize(input_tokens_ex))


Frase:  tf.Tensor(b'this picture seemed way to slanted, it\'s almost as bad as the drum beating of the right wing kooks who say everything is rosy in iraq. it paints a picture so unredeemable that i can\'t help but wonder about it\'s legitimacy and bias. also it seemed to meander from being about the murderous carnage of our troops to the lack of health care in the states for ptsd. to me the subject matter seemed confused, it only cared about portraying the military in a bad light, as a) an organzation that uses mind control to turn ordinary peace loving civilians into baby killers and b) an organization that once having used and spent the bodies of it\'s soldiers then discards them to the despotic bureacracy of the v.a. this is a legitimate argument, but felt off topic for me, almost like a movie in and of itself. i felt that "the war tapes" and "blood of my brother" were much more fair and let the viewer draw some conclusions of their own rather than be beaten over the head with the 

#Formatando o dataset
Aqui formataremos o dataset para a forma com a qual alimentaremos os modelos.

In [ ]:

def format_dataset(sentence, label):
    sentence = tokenizer(sentence)
    return ({"input_ids": sentence}, label)


def make_dataset(dataset):
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(512).prefetch(16).cache()


train_ds = make_dataset(train_ds)
val_ds = make_dataset(val_ds)
test_ds = make_dataset(test_ds)

#Construindo o modelo
Precisamos de uma camada embedding, uma camada que mapeia todos os tokens do imput em um vetor. Essa camada pode ser inicializada aleatoriamente. Também precisamos de uma camada embedding posicional, que codifica a ordem das palavras na sequência. O KerasNLP tem uma camada "keras_nlp.layers.TokenAndPositionEmbedding" que faz tudo isso para nós.

In [ ]:
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

fnet_classifier = keras.Model(input_ids, outputs, name="fnet_classifier")

#Treinando o modelo
Vamos usar a acurácia para monitorar o progresso do treinamento.

In [ ]:
fnet_classifier.summary()
fnet_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
fnet_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "fnet_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 128)        1985536   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 f_net_encoder (FNetEncoder)  (None, None, 128)        132224    
                                                                 
 f_net_encoder_1 (FNetEncode  (None, None, 128)        132224    
 r)                                                              
                                                                 
 f_net_encoder_2 (FNetEncode  (None, None, 128)        132224    
 r)                                                

Obtivemos uma acurácia de treino de 92,39% e uma acurácia de validação de 85,54%. O treinamento levou 81 segundos.

#Calculando a precisão do teste

In [ ]:
fnet_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)

391/391 [==============================] - 17s 27ms/step - loss: 0.3962 - accuracy: 0.8418


[0.39619317650794983, 0.841759979724884]

#Comparando com o Transformer
Aqui vamos comparar o modelo FNet com um modelo Transformer. Vamos manter todos os parâmetros iguais.

In [ ]:
NUM_HEADS = 2
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")


x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

transformer_classifier = keras.Model(input_ids, outputs, name="transformer_classifier")


transformer_classifier.summary()
transformer_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
transformer_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 128)        1985536   
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder (Transf  (None, None, 128)        198272    
 ormerEncoder)                                                   
                                                                 
 transformer_encoder_1 (Tran  (None, None, 128)        198272    
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_2 (Tran  (None, None, 1

Obtivemos uma acurácia de treino de 93,80% e uma acurácia de validação de 87,30%. O treinamento levou 140 segundos.

#Calculando a precisão do teste

In [ ]:
transformer_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)

391/391 [==============================] - 22s 56ms/step - loss: 0.4792 - accuracy: 0.8436


[0.4792218804359436, 0.8435999751091003]

#Sequence Length Grande
Aqui vamos implementar o Embedding A, com sequence length maior. No caso, 1024. 


**Todos os comentários feitos anteriormente servem para esta parte do trabalho também**

In [ ]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 1024
VOCAB_SIZE = 15000

EMBED_DIM = 128
INTERMEDIATE_DIM = 512

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42,
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42,
)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=BATCH_SIZE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
train_ds = train_ds.map(lambda x, y: (tf.strings.lower(x), y))
val_ds = val_ds.map(lambda x, y: (tf.strings.lower(x), y))
test_ds = test_ds.map(lambda x, y: (tf.strings.lower(x), y))

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])


b'an illegal immigrant resists the social support system causing dire consequences for many. well filmed and acted even though the story is a bit forced, yet the slow pacing really sets off the conclusion. the feeling of being lost in the big city is effectively conveyed. the little person lost in the big society is something to which we can all relate, but i cannot endorse going out of your way to see this movie.'
0
b"to get in touch with the beauty of this film pay close attention to the sound track, not only the music, but the way all sounds help to weave the imagery. how beautifully the opening scene leading to the expulsion of gino establishes the theme of moral ambiguity! note the way music introduces the characters as we are led inside giovanna's marriage. don't expect to find much here of the political life of italy in 1943. that's not what this is about. on the other hand, if you are susceptible to the music of images and sounds, you will be led into a word that reaches beyond

In [ ]:

def train_word_piece(ds, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # Tamanho do vocabulário de destino
        vocab_size=vocab_size,
        # Tokens reservados que devem ser incluídos no vocabulário 
        reserved_tokens=reserved_tokens,
        # Argumentos para `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    # Extrai amostras de texto (remove os rótulos).
    word_piece_ds = ds.unbatch().map(lambda x, y: x)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab


In [ ]:
reserved_tokens = ["[PAD]", "[UNK]"]
train_sentences = [element[0] for element in train_ds]
vocab = train_word_piece(train_ds, VOCAB_SIZE, reserved_tokens)

In [ ]:
print("Tokens: ", vocab[100:110])

Tokens:  ['in', 'this', 'that', 'was', 'as', 'for', 'movie', 'with', 'but', 'film']


In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

In [ ]:
input_sentence_ex = train_ds.take(1).get_single_element()[0][0]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Frase: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Texto recuperado após a destokenização: ", tokenizer.detokenize(input_tokens_ex))


Frase:  tf.Tensor(b'this picture seemed way to slanted, it\'s almost as bad as the drum beating of the right wing kooks who say everything is rosy in iraq. it paints a picture so unredeemable that i can\'t help but wonder about it\'s legitimacy and bias. also it seemed to meander from being about the murderous carnage of our troops to the lack of health care in the states for ptsd. to me the subject matter seemed confused, it only cared about portraying the military in a bad light, as a) an organzation that uses mind control to turn ordinary peace loving civilians into baby killers and b) an organization that once having used and spent the bodies of it\'s soldiers then discards them to the despotic bureacracy of the v.a. this is a legitimate argument, but felt off topic for me, almost like a movie in and of itself. i felt that "the war tapes" and "blood of my brother" were much more fair and let the viewer draw some conclusions of their own rather than be beaten over the head with the 

In [ ]:

def format_dataset(sentence, label):
    sentence = tokenizer(sentence)
    return ({"input_ids": sentence}, label)


def make_dataset(dataset):
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(512).prefetch(16).cache()


train_ds = make_dataset(train_ds)
val_ds = make_dataset(val_ds)
test_ds = make_dataset(test_ds)

In [ ]:
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

fnet_classifier = keras.Model(input_ids, outputs, name="fnet_classifier")

In [ ]:
fnet_classifier.summary()
fnet_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
fnet_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "fnet_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 128)        2051072   
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 f_net_encoder_3 (FNetEncode  (None, None, 128)        132224    
 r)                                                              
                                                                 
 f_net_encoder_4 (FNetEncode  (None, None, 128)        132224    
 r)                                                              
                                                                 
 f_net_encoder_5 (FNetEncode  (None, None, 128)    

Obtivemos uma acurácia de treino de 90,17% e uma acurácia de validação de 84,30%. O treinamento levou 141 segundos.

#Calculando a precisão do teste

In [ ]:
fnet_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)


391/391 [==============================] - 25s 49ms/step - loss: 0.4131 - accuracy: 0.8340


[0.41311198472976685, 0.8340399861335754]

#Comparando com o Transformer

In [ ]:
NUM_HEADS = 2
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")


x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

transformer_classifier = keras.Model(input_ids, outputs, name="transformer_classifier")


transformer_classifier.summary()
transformer_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
transformer_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 128)        2051072   
 g_3 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder_3 (Tran  (None, None, 128)        198272    
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_4 (Tran  (None, None, 128)        198272    
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_5 (Tran  (None, None, 1

Obtivemos uma acurácia de treino de 93,34% e uma acurácia de validação de 88,80%. O treinamento levou 378 segundos.

#Calculando a precisão do teste

In [ ]:
transformer_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)

391/391 [==============================] - 62s 159ms/step - loss: 0.3782 - accuracy: 0.8706


[0.37821200489997864, 0.8705999851226807]

# EMBEDDING B
A Embedding B tem um tamanho maior que a anterior.

# Sequence Lenght Pequeno


In [ ]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 512
VOCAB_SIZE = 15000

EMBED_DIM = 512
INTERMEDIATE_DIM = 1024

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42,
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42,
)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=BATCH_SIZE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
train_ds = train_ds.map(lambda x, y: (tf.strings.lower(x), y))
val_ds = val_ds.map(lambda x, y: (tf.strings.lower(x), y))
test_ds = test_ds.map(lambda x, y: (tf.strings.lower(x), y))

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])


b'an illegal immigrant resists the social support system causing dire consequences for many. well filmed and acted even though the story is a bit forced, yet the slow pacing really sets off the conclusion. the feeling of being lost in the big city is effectively conveyed. the little person lost in the big society is something to which we can all relate, but i cannot endorse going out of your way to see this movie.'
0
b"to get in touch with the beauty of this film pay close attention to the sound track, not only the music, but the way all sounds help to weave the imagery. how beautifully the opening scene leading to the expulsion of gino establishes the theme of moral ambiguity! note the way music introduces the characters as we are led inside giovanna's marriage. don't expect to find much here of the political life of italy in 1943. that's not what this is about. on the other hand, if you are susceptible to the music of images and sounds, you will be led into a word that reaches beyond

In [ ]:

def train_word_piece(ds, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # Tamanho do vocabulário de destino
        vocab_size=vocab_size,
        # Tokens reservados que devem ser incluídos no vocabulário 
        reserved_tokens=reserved_tokens,
        # Argumentos para `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    # Extrai amostras de texto (remove os rótulos).
    word_piece_ds = ds.unbatch().map(lambda x, y: x)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab


In [ ]:
reserved_tokens = ["[PAD]", "[UNK]"]
train_sentences = [element[0] for element in train_ds]
vocab = train_word_piece(train_ds, VOCAB_SIZE, reserved_tokens)

In [ ]:
print("Tokens: ", vocab[100:110])

Tokens:  ['in', 'this', 'that', 'was', 'as', 'for', 'movie', 'with', 'but', 'film']


In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

In [ ]:
input_sentence_ex = train_ds.take(1).get_single_element()[0][0]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Frase: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Texto recuperado após a destokenização: ", tokenizer.detokenize(input_tokens_ex))


Frase:  tf.Tensor(b'this picture seemed way to slanted, it\'s almost as bad as the drum beating of the right wing kooks who say everything is rosy in iraq. it paints a picture so unredeemable that i can\'t help but wonder about it\'s legitimacy and bias. also it seemed to meander from being about the murderous carnage of our troops to the lack of health care in the states for ptsd. to me the subject matter seemed confused, it only cared about portraying the military in a bad light, as a) an organzation that uses mind control to turn ordinary peace loving civilians into baby killers and b) an organization that once having used and spent the bodies of it\'s soldiers then discards them to the despotic bureacracy of the v.a. this is a legitimate argument, but felt off topic for me, almost like a movie in and of itself. i felt that "the war tapes" and "blood of my brother" were much more fair and let the viewer draw some conclusions of their own rather than be beaten over the head with the 

In [ ]:

def format_dataset(sentence, label):
    sentence = tokenizer(sentence)
    return ({"input_ids": sentence}, label)


def make_dataset(dataset):
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(512).prefetch(16).cache()


train_ds = make_dataset(train_ds)
val_ds = make_dataset(val_ds)
test_ds = make_dataset(test_ds)

In [ ]:
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

fnet_classifier = keras.Model(input_ids, outputs, name="fnet_classifier")

In [ ]:
fnet_classifier.summary()
fnet_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
fnet_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "fnet_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 512)        7942144   
 g_4 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 f_net_encoder_6 (FNetEncode  (None, None, 512)        1052160   
 r)                                                              
                                                                 
 f_net_encoder_7 (FNetEncode  (None, None, 512)        1052160   
 r)                                                              
                                                                 
 f_net_encoder_8 (FNetEncode  (None, None, 512)    

#Calculando a precisão do teste

In [ ]:
fnet_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)


391/391 [==============================] - 51s 110ms/step - loss: 0.3989 - accuracy: 0.8389


[0.39893269538879395, 0.8388800024986267]

#Comparando com o Transformer

In [ ]:
NUM_HEADS = 2
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")


x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

transformer_classifier = keras.Model(input_ids, outputs, name="transformer_classifier")


transformer_classifier.summary()
transformer_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
transformer_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 512)        7942144   
 g_5 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder_6 (Tran  (None, None, 512)        2102784   
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_7 (Tran  (None, None, 512)        2102784   
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_8 (Tran  (None, None, 5

#Calculando a precisão do teste

In [ ]:
transformer_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)

391/391 [==============================] - 81s 207ms/step - loss: 0.6947 - accuracy: 0.5000


[0.6946806907653809, 0.5]

# Sequence Lenght Grande

In [ ]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 1024
VOCAB_SIZE = 15000

EMBED_DIM = 512
INTERMEDIATE_DIM = 1024

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42,
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42,
)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=BATCH_SIZE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
train_ds = train_ds.map(lambda x, y: (tf.strings.lower(x), y))
val_ds = val_ds.map(lambda x, y: (tf.strings.lower(x), y))
test_ds = test_ds.map(lambda x, y: (tf.strings.lower(x), y))

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])


b'an illegal immigrant resists the social support system causing dire consequences for many. well filmed and acted even though the story is a bit forced, yet the slow pacing really sets off the conclusion. the feeling of being lost in the big city is effectively conveyed. the little person lost in the big society is something to which we can all relate, but i cannot endorse going out of your way to see this movie.'
0
b"to get in touch with the beauty of this film pay close attention to the sound track, not only the music, but the way all sounds help to weave the imagery. how beautifully the opening scene leading to the expulsion of gino establishes the theme of moral ambiguity! note the way music introduces the characters as we are led inside giovanna's marriage. don't expect to find much here of the political life of italy in 1943. that's not what this is about. on the other hand, if you are susceptible to the music of images and sounds, you will be led into a word that reaches beyond

In [ ]:

def train_word_piece(ds, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # Tamanho do vocabulário de destino
        vocab_size=vocab_size,
        # Tokens reservados que devem ser incluídos no vocabulário 
        reserved_tokens=reserved_tokens,
        # Argumentos para `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    # Extrai amostras de texto (remove os rótulos).
    word_piece_ds = ds.unbatch().map(lambda x, y: x)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab


In [ ]:
reserved_tokens = ["[PAD]", "[UNK]"]
train_sentences = [element[0] for element in train_ds]
vocab = train_word_piece(train_ds, VOCAB_SIZE, reserved_tokens)

In [ ]:
print("Tokens: ", vocab[100:110])

Tokens:  ['in', 'this', 'that', 'was', 'as', 'for', 'movie', 'with', 'but', 'film']


In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

In [ ]:
input_sentence_ex = train_ds.take(1).get_single_element()[0][0]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Frase: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Texto recuperado após a destokenização: ", tokenizer.detokenize(input_tokens_ex))


Frase:  tf.Tensor(b'this picture seemed way to slanted, it\'s almost as bad as the drum beating of the right wing kooks who say everything is rosy in iraq. it paints a picture so unredeemable that i can\'t help but wonder about it\'s legitimacy and bias. also it seemed to meander from being about the murderous carnage of our troops to the lack of health care in the states for ptsd. to me the subject matter seemed confused, it only cared about portraying the military in a bad light, as a) an organzation that uses mind control to turn ordinary peace loving civilians into baby killers and b) an organization that once having used and spent the bodies of it\'s soldiers then discards them to the despotic bureacracy of the v.a. this is a legitimate argument, but felt off topic for me, almost like a movie in and of itself. i felt that "the war tapes" and "blood of my brother" were much more fair and let the viewer draw some conclusions of their own rather than be beaten over the head with the 

In [ ]:

def format_dataset(sentence, label):
    sentence = tokenizer(sentence)
    return ({"input_ids": sentence}, label)


def make_dataset(dataset):
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(512).prefetch(16).cache()


train_ds = make_dataset(train_ds)
val_ds = make_dataset(val_ds)
test_ds = make_dataset(test_ds)

In [ ]:
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim=INTERMEDIATE_DIM)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

fnet_classifier = keras.Model(input_ids, outputs, name="fnet_classifier")

In [ ]:
fnet_classifier.summary()
fnet_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
fnet_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "fnet_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 512)        8204288   
 g_6 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 f_net_encoder_9 (FNetEncode  (None, None, 512)        1052160   
 r)                                                              
                                                                 
 f_net_encoder_10 (FNetEncod  (None, None, 512)        1052160   
 er)                                                             
                                                                 
 f_net_encoder_11 (FNetEncod  (None, None, 512)    

#Calculando a precisão do teste

In [ ]:
fnet_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)


391/391 [==============================] - 91s 213ms/step - loss: 0.3777 - accuracy: 0.8372


[0.37769144773483276, 0.8371999859809875]

#Comparando com o Transformer

In [ ]:
NUM_HEADS = 2
input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")


x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(input_ids)

x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)


x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

transformer_classifier = keras.Model(input_ids, outputs, name="transformer_classifier")


transformer_classifier.summary()
transformer_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
transformer_classifier.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 512)        8204288   
 g_7 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder_9 (Tran  (None, None, 512)        2102784   
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_10 (Tra  (None, None, 512)        2102784   
 nsformerEncoder)                                                
                                                                 
 transformer_encoder_11 (Tra  (None, None, 5

#Calculando a precisão do teste

In [ ]:
transformer_classifier.evaluate(test_ds, batch_size=BATCH_SIZE)

391/391 [==============================] - 195s 500ms/step - loss: 0.7004 - accuracy: 0.5002


[0.7003594636917114, 0.5001599788665771]

#***CONCLUSÃO***

**PARA A EMBEDDING A**

*Sequence length pequena*

Analisando ambas arquiteturas, podemos ver que o FNet diminui muito o tempo de execução apenas perdendo um pouco da acurácia no geral.

Podemos concluir que, no caso da Embedding A com sequence length pequena, a melhor arquitetura foi o FNet, já que seu tempo de execução é 1,7 vezes mais rápida com uma pequena diferença na acurácia geral (1,4%).

*Sequence length grande*

Analisando ambas as arquiteturas para uma sequence length maior, podemos observar que o tempo de execução do transformer foi muito maior que do fnet (378 segundos e 141 segundos, respectivamente), perdendo um pouco da acurácia.

Podemos concluir que, no caso da Embedding A com sequence length grande, a melhor arquitetura foi o FNet também, já que seu tempo de execução é 2,6 vezes mais rápida que o transformer com uma pequena diferença na acurácia geral (3%).

Comparando os sequence length grande e pequeno e as arquiteturas, podemos observar que quando o sequence length é grande, o FNet tem uma maior diferença de tempo em relação ao Transformer, porém uma perda maior na acurácia.

**PARA A EMBEDDING B**

*Sequence length pequena*

Analisando ambas arquiteturas, podemos observar que o FNet tem um tempo de execução menor que o Transformer porém a acurácia no Transformer diminui quase pela metade (acurácia FNet: 89,31%; acurácia Transformer: 49,47%).

Podemos concluir que a melhor arquitetura aqui também foi o FNet, já que tanto a acurácia quanto o tempo de execução foram melhores (acurácia maior e tempo menor).

*Sequence length grande*

Analisando ambas as arquiteturas para uma sequence length maior, podemos observar que o tempo de execução do transformer foi muito maior que do fnet (1349 segundos e 630 segundos, respectivamente), perdendo muito da acurácia (FNet teve uma acurácia de 88,56% enquanto o Transformer obteve uma acurácia de 50,76%).

Podemos concluir que, no caso da Embedding B com sequence length grande, a melhor arquitetura foi o FNet também, já que seu tempo de execução é 1,8 vezes mais rápida que o transformer com uma grande diferença na acurácia geral (39,84%).

# ***CONCLUSÃO GERAL***

Em relação aos embeddings, quando aumentamos o tamanho o tempo de execução também aumenta consideravelmente e há uma perda de acurácia para ambos os casos.

Por fim, podemos dizer que, comparando entre estas duas arquiteturas implementadas neste trabalho para o problema de classificação de texto, o modelo FNet é a melhor escolha, uma vez que se saiu melhor na maioria dos cenários quando levamos em conta acurácia em relação ao tempo de execução.